In [17]:
import pygame
import random
import math

# Inicializa todos os módulos do Pygame
pygame.init()

# Define a largura e altura da janela de exibição
WIDTH, HEIGHT = 900, 900
# Cria uma janela de exibição com as dimensões especificadas
WIN = pygame.display.set_mode((WIDTH, HEIGHT))
# Define o título da janela de exibição
pygame.display.set_caption("Simulação de Sistema Solar")

# Definição de cores em formato RGB
WHITE = (255, 255, 255)    # Cor branca
YELLOW = (255, 223, 0)     # Cor amarela (para o Sol)
BLUE = (65, 105, 225)      # Cor azul (para a Terra)
RED = (205, 92, 92)        # Cor vermelha (para Marte)
GREY = (169, 169, 169)     # Cor cinza (para Mercúrio)
ORANGE = (255, 165, 0)     # Cor laranja (para Vênus)

# Define a fonte para o texto exibido na simulação
FONT = pygame.font.SysFont("sans-serif", 24)

def draw_stars():
    """
    Função que desenha estrelas no fundo da tela. As posições, cores e tamanhos das estrelas
    são gerados aleatoriamente, mas a seed é definida para garantir a mesma distribuição
    em cada execução.
    """
    random.seed(42) 
    # Gera uma lista de estrelas com suas respectivas cores, posições e tamanhos
    stars = [((random.randint(150, 200), random.randint(150, 200), random.randint(150, 200)), 
              (random.randint(1, WIDTH), random.randint(1, HEIGHT)), 
              random.randint(1, 2)) for _ in range(250)]
    
    for star in stars:
        pygame.draw.circle(WIN, star[0], star[1], star[2])

class Planet:
    """
    Classe que representa um planeta no sistema solar.

    Atributos:
        AU (float): Unidade Astronômica (distância média entre a Terra e o Sol) em metros.
        G (float): Constante gravitacional.
        SCALE (float): Escala para converter metros em pixels.
        TIMESTEP (int): Passo de tempo em segundos (um dia em simulação).

        x (float): Posição x do planeta em metros.
        y (float): Posição y do planeta em metros.
        radius (int): Raio do planeta em pixels.
        color (tuple): Cor do planeta em RGB.
        mass (float): Massa do planeta em quilogramas.
        name (str): Nome do planeta.

        orbit (list): Lista de pontos que representam a órbita do planeta.
        sun (bool): Indica se o planeta é o Sol.
        distance_to_sun (float): Distância do planeta ao Sol.
        x_vel (float): Velocidade na direção x.
        y_vel (float): Velocidade na direção y.
    """
    AU = 149.6e6 * 1000  # Unidade Astronômica (m)
    G = 6.67428e-11      # Constante Gravitacional (N*m^2/kg^2)
    SCALE = 250 / AU     # Escala (250 pixels = 1 AU)
    TIMESTEP = 3600 * 24 # Passo de tempo (um dia)

    def __init__(self, x, y, radius, color, mass, name):
        self.x = x
        self.y = y
        self.radius = radius
        self.color = color
        self.mass = mass
        self.name = name

        self.orbit = []
        self.sun = False
        self.distance_to_sun = 0

        self.x_vel = 0
        self.y_vel = 0

    def draw(self, win):
        """
        Desenha o planeta na tela e a sua órbita, se existente.

        Args:
            win (pygame.Surface): A superfície onde o planeta será desenhado.
        """
        # Converte as posições do planeta para o sistema de coordenadas da janela
        x = self.x * self.SCALE + WIDTH / 2
        y = self.y * self.SCALE + HEIGHT / 2

        # Desenha a órbita do planeta se existirem pontos suficientes
        if len(self.orbit) > 2:
            updated_points = []
            for point in self.orbit:
                x, y = point
                x = x * self.SCALE + WIDTH / 2
                y = y * self.SCALE + HEIGHT / 2
                updated_points.append((x, y))

            pygame.draw.aalines(win, self.color, False, updated_points, 2)

        # Desenha o planeta como um círculo
        pygame.draw.circle(win, self.color, (x, y), self.radius)

        # Se o planeta não for o Sol, exibe a distância até o Sol
        if not self.sun:
            distance_text = FONT.render(f"{round(self.distance_to_sun/self.AU, 3)}au", 1, WHITE)
            win.blit(distance_text, (x - distance_text.get_width()/2, y - distance_text.get_height()/2))
        
        # Desenha o nome do planeta acima dele
        name_text = FONT.render(self.name, 1, WHITE)
        win.blit(name_text, (x - name_text.get_width()/2, y - self.radius - 20))

    def attraction(self, other):
        """
        Calcula a força de atração gravitacional exercida por outro planeta.

        Args:
            other (Planet): Outro planeta que exerce força gravitacional.

        Returns:
            tuple: Componentes x e y da força.
        """
        other_x, other_y = other.x, other.y
        distance_x = other_x - self.x
        distance_y = other_y - self.y
        distance = math.sqrt(distance_x ** 2 + distance_y ** 2)

        # Se o outro planeta for o Sol, armazena a distância até ele
        if other.sun:
            self.distance_to_sun = distance

        # Calcula a força gravitacional
        force = self.G * self.mass * other.mass / distance**2
        theta = math.atan2(distance_y, distance_x)
        force_x = math.cos(theta) * force
        force_y = math.sin(theta) * force
        return force_x, force_y

    def update_position(self, planets):
        """
        Atualiza a posição do planeta com base nas forças exercidas por outros planetas.

        Args:
            planets (list): Lista de todos os planetas no sistema.
        """
        total_fx = total_fy = 0
        for planet in planets:
            if self == planet:
                continue

            # Soma as forças gravitacionais de todos os planetas
            fx, fy = self.attraction(planet)
            total_fx += fx
            total_fy += fy

        # Atualiza as velocidades com base na força resultante
        self.x_vel += total_fx / self.mass * self.TIMESTEP
        self.y_vel += total_fy / self.mass * self.TIMESTEP

        # Atualiza as posições com base nas velocidades
        self.x += self.x_vel * self.TIMESTEP
        self.y += self.y_vel * self.TIMESTEP

        # Armazena a nova posição na órbita
        self.orbit.append((self.x, self.y))


def main():
    """
    Função principal que executa a simulação do sistema solar.
    """
    run = True
    clock = pygame.time.Clock() 

    # Criação dos planetas do sistema solar
    sun = Planet(0, 0, 30, YELLOW, 1.98892 * 10**30, "Sol")
    sun.sun = True
    
    mercury = Planet(0.387 * Planet.AU, 0, 8, GREY, 3.30 * 10**23, "Mercúrio")
    mercury.y_vel = -47.4 * 1000  # Velocidade orbital inicial de Mercúrio

    venus = Planet(0.723 * Planet.AU, 0, 14, ORANGE, 4.8685 * 10**24, "Vênus")
    venus.y_vel = -35.02 * 1000  # Velocidade orbital inicial de Vênus
    
    earth = Planet(1 * Planet.AU, 0, 16, BLUE, 5.9742 * 10**24, "Terra")
    earth.y_vel = -29.783 * 1000  # Velocidade orbital inicial da Terra

    mars = Planet(1.524 * Planet.AU, 0, 12, RED, 6.39 * 10**23, "Marte")
    mars.y_vel = -24.077 * 1000  # Velocidade orbital inicial de Marte

    # Lista de planetas que serão simulados
    planets = [sun, mercury, venus, earth, mars]

    i = 0  # Contador para salvar screenshots
    while run:
        clock.tick(20)  # Controla a taxa de atualização para 20 quadros por segundo

        for event in pygame.event.get():
            if event.type == pygame.QUIT:  # Verifica se a janela foi fechada
                run = False

        WIN.fill((0, 0, 0))  # Preenche o fundo da janela com a cor preta
        draw_stars()  # Desenha as estrelas no fundo

        for planet in planets:
            planet.update_position(planets)  
            planet.draw(WIN)  

        # Salva uma screenshot a cada 10 iterações do loop
        if i % 10 == 0:
            pygame.image.save(WIN, f"screenshot_{i}.png")
        
        i += 1
        pygame.display.update()  

    pygame.quit()  

if __name__ == "__main__":
    main()
